1. 전체 3년치 데이터 전부 전처리해서 새 전체 데이터셋 만든다
2. 그걸 다시 지역별로 나눠서 df_sambangdong ... 만든다
3. 각 지역별로 년, 월, 시간대별로 그래프 그리고 상관분석 한다
4. 전체 지역 지도 위에 한눈에 볼 수 있게 분석한다
5. 데이터셋들을 갖고 지역별, 전체 인공지능 모델 예측한다


숙제
1. 보고서의 목차 만들어오기 : 장제목 절제목 결론 → 선행연구 베껴와서 말만 바꾸기 
2. 한 지역에 대한 그래프 샘플 - 9개 상관분석 마지막에 지도 지역별 한 슬라이드로

In [12]:
import pandas as pd

# 파일 경로 설정
file_21 = '/Users/hayden/Desktop/provincial council internship/2021년전체측정소자료.csv'
file_22 = '/Users/hayden/Desktop/provincial council internship/2022년전체측정소자료.csv'
file_23 = '/Users/hayden/Desktop/provincial council internship/2023년전체측정소자료.csv'

# 각각의 CSV 파일을 데이터프레임으로 읽기
df_21 = pd.read_csv(file_21)
df_22 = pd.read_csv(file_22)
df_23 = pd.read_csv(file_23)

# 데이터프레임을 리스트에 담기
dfs = [df_21, df_22, df_23]

# concat 함수를 사용하여 합치기
df = pd.concat(dfs, ignore_index=True)

# 결과 확인
print(df.head())  # 합쳐진 데이터프레임의 첫 부분 출력


       MSR_DT STTN_NM  PM25  PM10     O3   CO    SO2    NO2     NO    NOX
0  2021010101     명서동  11.0  22.0  0.022  0.5  0.003  0.023  0.009  0.032
1  2021010102     명서동  12.0  16.0  0.028  0.4  0.003  0.015  0.007  0.022
2  2021010103     명서동  13.0  21.0  0.031  0.4  0.003  0.013  0.006  0.019
3  2021010104     명서동  15.0  19.0  0.028  0.4  0.003  0.013  0.006  0.019
4  2021010105     명서동  15.0  22.0  0.024  0.4  0.003  0.015  0.006  0.021


In [13]:
# 'STTN_NM'이 '이동차', '반송로', '부산항'인 행을 삭제
df = df[~df['STTN_NM'].isin(['이동차', '반송로', '부산항'])]

# 'NO' 및 'NOX' 컬럼을 삭제
df.drop(columns=['NO', 'NOX'], inplace=True)

# 결과 출력
print(df)

# 38만개 * 3 = 114만개 -> 필요없는 데이터 제거 후 106만개 

             MSR_DT STTN_NM  PM25  PM10     O3   CO    SO2    NO2
0        2021010101     명서동  11.0  22.0  0.022  0.5  0.003  0.023
1        2021010102     명서동  12.0  16.0  0.028  0.4  0.003  0.015
2        2021010103     명서동  13.0  21.0  0.031  0.4  0.003  0.013
3        2021010104     명서동  15.0  19.0  0.028  0.4  0.003  0.013
4        2021010105     명서동  15.0  22.0  0.024  0.4  0.003  0.015
...             ...     ...   ...   ...    ...  ...    ...    ...
1117136  2023123120     마산항   0.0  11.0  0.037  0.4  0.001  0.013
1117137  2023123121     마산항  15.0  15.0  0.036  0.4  0.001  0.012
1117138  2023123122     마산항   0.0  14.0  0.035  0.4  0.001  0.012
1117139  2023123123     마산항   0.0  11.0  0.032  0.4  0.001  0.014
1117140  2023123124     마산항   0.0  16.0  0.031  0.4  0.001  0.013

[1069978 rows x 8 columns]


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1069978 entries, 0 to 1117140
Data columns (total 8 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   MSR_DT   1069978 non-null  int64  
 1   STTN_NM  1069978 non-null  object 
 2   PM25     997821 non-null   float64
 3   PM10     1003510 non-null  float64
 4   O3       1018389 non-null  float64
 5   CO       1010370 non-null  float64
 6   SO2      1016054 non-null  float64
 7   NO2      1009708 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 73.5+ MB


In [18]:
# 각 컬럼의 NULL 값 개수 확인
null_counts = df.isnull().sum()

# 특정 컬럼의 NULL 값 개수 확인
PM25_null_count = df['PM25'].isnull().sum()
PM10_null_count = df['PM10'].isnull().sum()
O3_null_count = df['O3'].isnull().sum()
CO_null_count = df['CO'].isnull().sum()
SO2_null_count = df['SO2'].isnull().sum()
NO2_null_count = df['NO2'].isnull().sum()

print("PM25 컬럼의 NULL 값 개수:", PM25_null_count)
print("PM10 컬럼의 NULL 값 개수:", PM10_null_count)
print("O3 컬럼의 NULL 값 개수:", O3_null_count)
print("CO 컬럼의 NULL 값 개수:", CO_null_count)
print("SO2 컬럼의 NULL 값 개수:", SO2_null_count)
print("NO2 컬럼의 NULL 값 개수:", NO2_null_count)

# 전체 데이터프레임의 NULL 값 개수 확인
total_null_count = df.isnull().sum().sum()
print("전체 데이터프레임의 NULL 값 개수:", total_null_count)



PM25 컬럼의 NULL 값 개수: 72157
PM10 컬럼의 NULL 값 개수: 66468
O3 컬럼의 NULL 값 개수: 51589
CO 컬럼의 NULL 값 개수: 59608
SO2 컬럼의 NULL 값 개수: 53924
NO2 컬럼의 NULL 값 개수: 60270
전체 데이터프레임의 NULL 값 개수: 364016


In [19]:
df.describe()

,MSR_DT,PM25,PM10,O3,CO,SO2,NO2
count,1.069978e+06,997821.000000,1.003510e+06,1.018389e+06,1.010370e+06,1.016054e+06,1.009708e+06
mean,2.022111e+09,15.326221,2.853739e+01,3.322246e-02,4.081526e-01,2.933830e-03,1.372420e-02
std,8.155589e+05,10.662294,2.781388e+01,1.854769e-02,1.606304e-01,1.318935e-03,1.024843e-02
min,2.021010e+09,-1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.021102e+09,8.000000,1.500000e+01,1.900000e-02,3.000000e-01,2.000000e-03,7.000000e-03
50%,2.022080e+09,13.000000,2.300000e+01,3.200000e-02,4.000000e-01,3.000000e-03,1.000000e-02
75%,2.023042e+09,20.000000,3.400000e+01,4.400000e-02,5.000000e-01,3.000000e-03,1.700000e-02
max,2.023123e+09,290.000000,1.260000e+03,2.200000e-01,5.120000e+00,1.520000e-01,1.080000e-01


In [20]:
import pandas as pd
import numpy as np

# 데이터프레임 로드
# df = pd.read_csv('/content/drive/MyDrive/2023년전체측정소자료수정.csv', encoding='cp949')



# 결측치가 있는 열을 선형 보간으로 채우기
for col in df.columns:
    # if df[col].dtype != 'datetime64[ns]':
    df[col] = df[col].interpolate(method='linear', limit_direction='both')

# DataFrame 복사본 생성 및 NULL 값 개수 확인
df_copy = df.copy()

# 각 컬럼의 NULL 값 개수 확인
null_counts = df_copy.isnull().sum()

# 특정 컬럼의 NULL 값 개수 확인
PM25_null_count = df_copy['PM25'].isnull().sum()
PM10_null_count = df_copy['PM10'].isnull().sum()
O3_null_count = df_copy['O3'].isnull().sum()
CO_null_count = df_copy['CO'].isnull().sum()
SO2_null_count = df_copy['SO2'].isnull().sum()
NO2_null_count = df_copy['NO2'].isnull().sum()

print("PM25 컬럼의 NULL 값 개수:", PM25_null_count)
print("PM10 컬럼의 NULL 값 개수:", PM10_null_count)
print("O3 컬럼의 NULL 값 개수:", O3_null_count)
print("CO 컬럼의 NULL 값 개수:", CO_null_count)
print("SO2 컬럼의 NULL 값 개수:", SO2_null_count)
print("NO2 컬럼의 NULL 값 개수:", NO2_null_count)

# 전체 데이터프레임의 NULL 값 개수 확인
total_null_count = df_copy.isnull().sum().sum()
print("전체 데이터프레임의 NULL 값 개수:", total_null_count)

/var/folders/6k/r_smdd550yddqpmrhzs0gqyh0000gn/T/ipykernel_47733/2312931886.py:12: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df[col] = df[col].interpolate(method='linear', limit_direction='both')


PM25 컬럼의 NULL 값 개수: 0
PM10 컬럼의 NULL 값 개수: 0
O3 컬럼의 NULL 값 개수: 0
CO 컬럼의 NULL 값 개수: 0
SO2 컬럼의 NULL 값 개수: 0
NO2 컬럼의 NULL 값 개수: 0
전체 데이터프레임의 NULL 값 개수: 0


In [22]:
df = df_copy
df.isnull().sum()

MSR_DT     0
STTN_NM    0
PM25       0
PM10       0
O3         0
CO         0
SO2        0
NO2        0
dtype: int64

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1069978 entries, 0 to 1117140
Data columns (total 8 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   MSR_DT   1069978 non-null  int64  
 1   STTN_NM  1069978 non-null  object 
 2   PM25     1069978 non-null  float64
 3   PM10     1069978 non-null  float64
 4   O3       1069978 non-null  float64
 5   CO       1069978 non-null  float64
 6   SO2      1069978 non-null  float64
 7   NO2      1069978 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 73.5+ MB


In [24]:
df_sambang = df[df['STTN_NM'] == '삼방동']
df_sambang.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26280 entries, 148920 to 881424
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   MSR_DT   26280 non-null  int64  
 1   STTN_NM  26280 non-null  object 
 2   PM25     26280 non-null  float64
 3   PM10     26280 non-null  float64
 4   O3       26280 non-null  float64
 5   CO       26280 non-null  float64
 6   SO2      26280 non-null  float64
 7   NO2      26280 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 1.8+ MB


In [25]:
df_sambang

,MSR_DT,STTN_NM,PM25,PM10,O3,CO,SO2,NO2
148920,2021010101,삼방동,10.0,18.0,0.0290,0.30,0.0020,0.0070
148921,2021010102,삼방동,7.0,18.0,0.0260,0.30,0.0020,0.0070
148922,2021010103,삼방동,8.0,24.0,0.0240,0.30,0.0030,0.0090
148923,2021010104,삼방동,7.0,25.0,0.0220,0.30,0.0020,0.0100
148924,2021010105,삼방동,6.0,20.0,0.0180,0.30,0.0020,0.0130
...,...,...,...,...,...,...,...,...
881420,2023123120,삼방동,8.0,15.0,0.0384,0.45,0.0018,0.0107
881421,2023123121,삼방동,8.0,17.0,0.0376,0.44,0.0018,0.0103
881422,2023123122,삼방동,13.0,19.0,0.0325,0.45,0.0019,0.0125
881423,2023123123,삼방동,10.0,23.0,0.0296,0.46,0.0017,0.0131


In [26]:
import pandas as pd
import matplotlib.pyplot as plt

# MSR_DT를 datetime 형식으로 변환
df_sambang['MSR_DT'] = pd.to_datetime(df_sambang['MSR_DT'], format='%Y%m%d%H', errors='coerce')

# 연도, 월, 시간 추출
df_sambang['Year'] = df_sambang['MSR_DT'].dt.year
df_sambang['Month'] = df_sambang['MSR_DT'].dt.month
df_sambang['Hour'] = df_sambang['MSR_DT'].dt.hour


/var/folders/6k/r_smdd550yddqpmrhzs0gqyh0000gn/T/ipykernel_47733/3697071976.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sambang['MSR_DT'] = pd.to_datetime(df_sambang['MSR_DT'], format='%Y%m%d%H', errors='coerce')
/var/folders/6k/r_smdd550yddqpmrhzs0gqyh0000gn/T/ipykernel_47733/3697071976.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sambang['Year'] = df_sambang['MSR_DT'].dt.year
/var/folders/6k/r_smdd550yddqpmrhzs0gqyh0000gn/T/ipykernel_47733/3697071976.py:9: SettingWithCopyWarning: 
A v

In [27]:
# 연도별 평균 계산
yearly_avg = df_sambang.groupby('Year').mean()

# 연도별 그래프
plt.figure(figsize=(12, 6))
yearly_avg['PM25'].plot(kind='bar', color='skyblue')
plt.title('Yearly Average PM2.5')
plt.xlabel('Year')
plt.ylabel('Average PM2.5')
plt.show()


TypeError: agg function failed [how->mean,dtype->object]

In [28]:
print(df_sambang.dtypes) # 정수가떠야 하는데 Null 값이 있어서 float으로 처리


MSR_DT     datetime64[ns]
STTN_NM            object
PM25              float64
PM10              float64
O3                float64
CO                float64
SO2               float64
NO2               float64
Year              float64
Month             float64
Hour              float64
dtype: object


In [29]:
df_sambang['Year'] = df_sambang['Year'].astype(int)
df_sambang['Month'] = df_sambang['Month'].astype(int)
df_sambang['Hour'] = df_sambang['Hour'].astype(int)


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [30]:
df_sambang['Hour']


148920     1.0
148921     2.0
148922     3.0
148923     4.0
148924     5.0
          ... 
881420    20.0
881421    21.0
881422    22.0
881423    23.0
881424     NaN
Name: Hour, Length: 26280, dtype: float64

In [31]:
df_sambang['Year']


148920    2021.0
148921    2021.0
148922    2021.0
148923    2021.0
148924    2021.0
           ...  
881420    2023.0
881421    2023.0
881422    2023.0
881423    2023.0
881424       NaN
Name: Year, Length: 26280, dtype: float64

In [32]:
df_sambang['Month']


148920     1.0
148921     1.0
148922     1.0
148923     1.0
148924     1.0
          ... 
881420    12.0
881421    12.0
881422    12.0
881423    12.0
881424     NaN
Name: Month, Length: 26280, dtype: float64

In [34]:
 # 인덱스가 881424이고 Month 열 값이 NaN인 행 확인
index = 881424
nan_month_row = df_sambang.loc[index]

print(nan_month_row)


MSR_DT        NaT
STTN_NM       삼방동
PM25         14.0
PM10         23.0
O3         0.0268
CO           0.46
SO2        0.0018
NO2        0.0132
Year          NaN
Month         NaN
Hour          NaN
Name: 881424, dtype: object


In [37]:
# 'Hour' 열에서 NaN 값을 정수 24로 대체
df_sambang['Hour'].fillna(24, inplace=True)
df_sambang['Hour']



/var/folders/6k/r_smdd550yddqpmrhzs0gqyh0000gn/T/ipykernel_47733/1488945788.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sambang['Hour'].fillna(24, inplace=True)


148920     1.0
148921     2.0
148922     3.0
148923     4.0
148924     5.0
          ... 
881420    20.0
881421    21.0
881422    22.0
881423    23.0
881424    24.0
Name: Hour, Length: 26280, dtype: float64

In [38]:
# Year 열에서 NaN 값을 가진 행 확인
nan_year_rows = df_sambang[df_sambang['Year'].isnull()]
print("NaN values in Year column:")
print(nan_year_rows)

# Month 열에서 NaN 값을 가진 행 확인
nan_month_rows = df_sambang[df_sambang['Month'].isnull()]
print("\nNaN values in Month column:")
print(nan_month_rows)


NaN values in Year column:
       MSR_DT STTN_NM  PM25  PM10      O3    CO     SO2     NO2  Year  Month  \
148943    NaT     삼방동   9.0  28.0  0.0130  0.40  0.0030  0.0270   NaN    NaN   
148967    NaT     삼방동  11.0  33.0  0.0210  0.50  0.0030  0.0180   NaN    NaN   
148991    NaT     삼방동   9.0  32.0  0.0330  0.30  0.0030  0.0080   NaN    NaN   
149015    NaT     삼방동  23.0  54.0  0.0040  0.60  0.0040  0.0450   NaN    NaN   
149039    NaT     삼방동   4.0  20.0  0.0310  0.30  0.0030  0.0110   NaN    NaN   
...       ...     ...   ...   ...     ...   ...     ...     ...   ...    ...   
881328    NaT     삼방동  18.0  47.0  0.0083  0.59  0.0035  0.0374   NaN    NaN   
881352    NaT     삼방동  37.0  63.0  0.0372  0.42  0.0027  0.0128   NaN    NaN   
881376    NaT     삼방동  34.0  59.0  0.0366  0.45  0.0023  0.0155   NaN    NaN   
881400    NaT     삼방동  47.0  69.0  0.0074  0.61  0.0038  0.0294   NaN    NaN   
881424    NaT     삼방동  14.0  23.0  0.0268  0.46  0.0018  0.0132   NaN    NaN   

        Hour

24시인 것들이 변환 실패로 NaT -> 그거 그대로 변환해서 또 NaN 뜨는 것이었다 세상에..

In [52]:
import pandas as pd
import matplotlib.pyplot as plt

# MSR_DT를 datetime 형식으로 변환
df_sambang['MSR_DT'] = pd.to_datetime(df_sambang['MSR_DT'], format='%Y%m%d%H', errors='coerce')

nat_rows = df[df['MSR_DT'].isnull()]

# NaT 값을 가진 행들의 인덱스를 리스트로 추출
nat_indices = nat_rows.index.tolist()
print("\nNaT 값을 가진 행들의 인덱스 리스트:")
print(nat_indices)



NaT 값을 가진 행들의 인덱스 리스트:
[]


/var/folders/6k/r_smdd550yddqpmrhzs0gqyh0000gn/T/ipykernel_47733/516037244.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sambang['MSR_DT'] = pd.to_datetime(df_sambang['MSR_DT'], format='%Y%m%d%H', errors='coerce')


In [51]:
# datetime 변환한 다음에 진행해야함
# NaT 값을 가진 행 탐색
nat_rows = df[df['MSR_DT'].isnull()]

# NaT 값을 가진 행들의 인덱스를 리스트로 추출
nat_indices = nat_rows.index.tolist()
print("\nNaT 값을 가진 행들의 인덱스 리스트:")
print(nat_indices)


NaT 값을 가진 행들의 인덱스 리스트:
[]


In [ ]:
# 인덱스에 해당하는 "MSR_DT"의 0, 1의 자리 숫자를 00으로 변환하는 함수  모든 24->00으로 변환
def change_last_two_digits_to_00(df2, indices):
    for index in indices:
        if index in df2.index:
            msr_dt = str(df2.at[index, 'MSR_DT'])
            msr_dt_new = msr_dt[:-2] + '00'
            df2.at[index, 'MSR_DT'] = int(msr_dt_new)
    return df2

# 함수 적용
df2 = change_last_two_digits_to_00(df2, nat_indices)

# 결과 출력
print(df2)

In [46]:
# 연도, 월, 시간 추출
df_sambang['Year'] = df_sambang['MSR_DT'].dt.year
df_sambang['Month'] = df_sambang['MSR_DT'].dt.month
df_sambang['Hour'] = df_sambang['MSR_DT'].dt.hour

AttributeError: Can only use .dt accessor with datetimelike values